In [9]:
# Importo linrerias necesarias
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [1]:
# %%
# Importo librerías necesarias
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# %%
print("NumPy version:", np.__version__)
print("TensorFlow version:", tf.__version__)

# %%
# Ruta donde están las imágenes
data_dir = "Converted_images"

# Definir listas para imágenes y etiquetas
images = []
labels = []
user_ids = []  # Añadido para identificar usuarios

# Recorrer las carpetas (cada carpeta representa un número del 0 al 9)
for digit in range(10):
    digit_path = os.path.join(data_dir, str(digit))  # Ruta de la carpeta del número
    if not os.path.exists(digit_path): 
        continue  # Si no existe, pasa al siguiente número
    
    for file in os.listdir(digit_path):
        img_path = os.path.join(digit_path, file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Leer en escala de grises
        img = cv2.resize(img, (28, 28))  # Redimensionar a 28x28
        images.append(img)
        labels.append(digit)  # La etiqueta es el número de la carpeta
        
        # Extraer el ID de usuario del nombre del archivo (los primeros 3 dígitos)
        try:
            # Extraer solo los dígitos del nombre del archivo
            digits_only = ''.join(c for c in file if c.isdigit())
            
            # El ID de usuario son los primeros 3 dígitos
            if len(digits_only) >= 3:
                user_id = int(digits_only[:3])
                user_ids.append(user_id)
            else:
                # Si no hay suficientes dígitos, usar un valor predeterminado
                user_ids.append(999)  # ID genérico
                
        except Exception as e:
            print(f"Error al procesar el ID de usuario para {file}: {e}")
            user_ids.append(999)  # ID genérico como respaldo

# Convertir a arrays de NumPy
images = np.array(images) / 255.0  # Normalizar valores entre 0 y 1
images = images.reshape(-1, 28, 28, 1)  # Agregar dimensión de canal (1 para blanco y negro)
labels = np.array(labels)
user_ids = np.array(user_ids)

# Dividir en conjuntos de entrenamiento y prueba (80%-20%)
X_train, X_test, y_train, y_test, usuarios_train, usuarios_test = train_test_split(
    images, labels, user_ids, test_size=0.8, random_state=42
)

# Guardar etiquetas antes de convertir a one-hot encoding
y_test_original = y_test.copy()

# Convertir etiquetas a one-hot encoding
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# %%
import os
data_dir = "Converted_images"

# Mostrar algunos archivos y sus IDs de usuario asignados para verificación
print("Verificación de asignación de IDs de usuario:")
user_id_samples = {}

for digit in range(10):
    digit_path = os.path.join(data_dir, str(digit))
    if os.path.exists(digit_path):
        files = os.listdir(digit_path)
        print(f"Número {digit}: {len(files)} archivos")
        
        # Mostrar hasta 3 ejemplos de nombres de archivo por dígito
        for file in files[:3]:
            try:
                # Extraer solo los dígitos del nombre del archivo
                digits_only = ''.join(c for c in file if c.isdigit())
                
                # El ID de usuario son los primeros 3 dígitos
                if len(digits_only) >= 3:
                    user_id = int(digits_only[:3])
                else:
                    user_id = 999  # ID genérico
                
                print(f"  - {file} -> Usuario ID: {user_id}")
                
                # Guardar para verificación de diversidad
                if user_id in user_id_samples:
                    user_id_samples[user_id] += 1
                else:
                    user_id_samples[user_id] = 1
                    
            except Exception as e:
                print(f"  - Error procesando {file}: {e}")

# Mostrar estadísticas de IDs de usuario encontrados
print(f"\nTotal de IDs de usuario distintos encontrados: {len(user_id_samples)}")
print(f"Distribución de muestras por usuario (primeros 10): {dict(list(user_id_samples.items())[:10])}")

# %%
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 clases (0 al 9)
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()  # Ver estructura de la red

# %%
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# %%
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scipy.interpolate import interp1d
from scipy.optimize import brentq
import os

# Crear directorios para guardar los resultados
os.makedirs('CNN/curvas_det_digitos', exist_ok=True)
os.makedirs('CNN/curvas_det_usuarios', exist_ok=True)

# Evaluar el modelo en conjunto de test
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Precisión global en test: {test_acc * 100:.2f}%')

# Obtener predicciones del modelo
y_probs = model.predict(X_test)
y_pred = np.argmax(y_probs, axis=1)

# ----- 1. Matriz de confusión general y accuracy por dígito -----
print("\n----- MATRIZ DE CONFUSIÓN GENERAL -----")
# Obtener versión original de y_test para la matriz de confusión
y_test_labels = np.argmax(y_test, axis=1)

# Verificar la distribución de usuarios y dígitos en el conjunto de test
print(f"Total de imágenes en test: {len(y_test_labels)}")
print(f"Distribución de dígitos en test: {np.unique(y_test_labels, return_counts=True)}")
print(f"Total de usuarios en test: {len(np.unique(usuarios_test))}")
print(f"Ejemplo de IDs de usuarios en test: {np.unique(usuarios_test)[:10]}")

conf_matrix = confusion_matrix(y_test_labels, y_pred)
plt.figure(figsize=(10, 8))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Matriz de Confusión')
plt.colorbar()
tick_marks = np.arange(10)
plt.xticks(tick_marks, range(10))
plt.yticks(tick_marks, range(10))
plt.xlabel('Predicción')
plt.ylabel('Valor Real')

# Añadir valores en cada celda
thresh = conf_matrix.max() / 2
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, format(conf_matrix[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if conf_matrix[i, j] > thresh else "black")
plt.tight_layout()
plt.savefig('CNN/matriz_confusion_general.png')
plt.close()

# Accuracy por dígito
print("\n----- ACCURACY POR DÍGITO -----")
accuracies_digito = []
for i in range(10):
    # Filtrar los valores para el dígito i
    indices_digito = (y_test_labels == i)
    if np.sum(indices_digito) > 0:
        acc_digito = accuracy_score(y_test_labels[indices_digito], y_pred[indices_digito])
        accuracies_digito.append(acc_digito)
        print(f'Accuracy para el dígito {i}: {acc_digito * 100:.2f}%')
    else:
        print(f'No hay muestras para el dígito {i}')
        accuracies_digito.append(0)

# Graficar accuracy por dígito
plt.figure(figsize=(10, 6))
plt.bar(range(10), [acc * 100 for acc in accuracies_digito])
plt.xlabel('Dígito')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy por Dígito')
plt.xticks(range(10))
plt.grid(True, axis='y')
plt.savefig('CNN/accuracy_por_digito.png')
plt.close()

# ----- 2. Accuracy por usuario -----
print("\n----- ACCURACY POR USUARIO -----")
# Crear DataFrame para resultados por usuario
resultados_usuario = pd.DataFrame({
    'usuario_num': usuarios_test,
    'real': y_test_labels,
    'predicho': y_pred,
    'correcto': y_test_labels == y_pred
})

# Calcular accuracy por usuario
accuracies_usuario = resultados_usuario.groupby('usuario_num')['correcto'].mean()
total_muestras = resultados_usuario.groupby('usuario_num')['correcto'].count()

# Crear DataFrame con resultados
df_accuracies = pd.DataFrame({
    'usuario_num': accuracies_usuario.index,
    'accuracy': accuracies_usuario.values,
    'total_muestras': total_muestras.values
})
df_accuracies['accuracy_porcentaje'] = df_accuracies['accuracy'] * 100

# Ordenar por accuracy (ascendente)
df_accuracies = df_accuracies.sort_values('accuracy')

# Mostrar usuarios con peor y mejor accuracy
print("\nUsuarios con peor accuracy:")
print(df_accuracies.head(5))
print("\nUsuarios con mejor accuracy:")
print(df_accuracies.tail(5))

# Guardar resultados
df_accuracies.to_csv('CNN/accuracy_por_usuario.csv', index=False)

# ----- 3. Función para calcular EER y curvas DET -----
def calculate_eer(y_true, y_score):
    """Versión robusta del cálculo de EER que maneja casos extremos"""
    # Verificar variabilidad en los puntajes
    if len(np.unique(y_score)) <= 1:
        return None, None, None, None  # No se puede calcular EER
    
    # Verificar variabilidad en las etiquetas
    if len(np.unique(y_true)) <= 1:
        return None, None, None, None  # No se puede calcular EER
    
    # Añadir pequeña variación a puntajes iguales para evitar problemas numéricos
    if len(np.unique(y_score)) < len(y_score):
        y_score = y_score + np.random.normal(0, 1e-10, len(y_score))
    
    try:
        fpr, tpr, thresholds = roc_curve(y_true, y_score)
        fnr = 1 - tpr
        
        # Verificar si hay suficientes puntos para interpolación
        if len(fpr) <= 2:
            return None, None, fpr, fnr
        
        # Usar método más robusto para encontrar el EER
        eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr, kind='linear', bounds_error=False, 
                                              fill_value=(0,1))(x), 0., 1., xtol=1e-8)
        threshold = interp1d(fpr, thresholds, bounds_error=False)(eer)
        
        return eer, threshold, fpr, fnr
    except Exception as e:
        print(f"Error en cálculo robusto de EER: {e}")
        return None, None, None, None

def plot_det_curve(fpr, fnr, eer=None, label=None, title=None, filename=None):
    """Grafica la curva DET"""
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, fnr, 'b-', linewidth=2, label=label if label else 'Curva DET')
    plt.plot([0, 1], [0, 1], 'k--', label='EER line')
    
    # Marcar el punto EER si está disponible
    if eer is not None:
        idx_eer = np.argmin(np.abs(fpr - (1-fnr)))
        plt.plot(fpr[idx_eer], fnr[idx_eer], 'ro', markersize=8, label=f'EER = {eer:.4f}')
    
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('False Negative Rate (FNR)')
    plt.title(title if title else 'Curva DET')
    plt.legend(loc='lower right')
    plt.grid(True)
    
    if filename:
        plt.savefig(f"CNN/{filename}")
        plt.close()
    else:
        plt.show()

# ----- 4. Análisis DET y EER por dígito -----
print("\n----- ANÁLISIS DET Y EER POR DÍGITO -----")
eers_digito = []
digitos = []

# One-vs-Rest para cada dígito
for digito in range(10):
    # Preparar etiquetas binarias (1 para el dígito actual, 0 para los demás)
    y_true_bin = (y_test_labels == digito).astype(int)
    # Obtener probabilidades para el dígito actual
    y_score = y_probs[:, digito]
    
    try:
        # Calcular EER
        eer, threshold, fpr, fnr = calculate_eer(y_true_bin, y_score)
        
        if eer is not None:
            eers_digito.append(eer)
            digitos.append(digito)
            
            print(f"Dígito {digito}: EER = {eer:.4f}, Umbral = {threshold:.4f}")
            
            # Guardar curva DET para este dígito
            plot_det_curve(
                fpr, fnr, eer=eer,
                label=f"Dígito {digito} (EER={eer:.4f})",
                title=f"Curva DET para Dígito {digito}",
                filename=f"curvas_det_digitos/det_digito_{digito}.png"
            )
        else:
            print(f"No se pudo calcular EER para Dígito {digito}")
    except Exception as e:
        print(f"Error al calcular EER para Dígito {digito}: {e}")

# Ordenar EER por dígito (solo si hay datos)
if eers_digito:
    df_eer_digito = pd.DataFrame({
        'digito': digitos,
        'eer': eers_digito
    })
    df_eer_digito = df_eer_digito.sort_values('eer')
    df_eer_digito.to_csv('CNN/eer_por_digito_ordenado.csv', index=False)

    # Graficar EER por dígito (ordenado)
    plt.figure(figsize=(10, 6))
    plt.bar(df_eer_digito['digito'], df_eer_digito['eer'])
    plt.xlabel('Dígito')
    plt.ylabel('Equal Error Rate (EER)')
    plt.title('EER por Dígito (ordenado de menor a mayor)')
    plt.xticks(range(10))
    plt.grid(True, axis='y')
    plt.savefig('CNN/eer_por_digito_ordenado.png')
    plt.close()

# ----- 5. Análisis DET y EER por usuario -----
print("\n----- ANÁLISIS DET Y EER POR USUARIO -----")
eers_usuario = []
usuarios_analizados = []

# Obtener usuarios únicos en test y su frecuencia
usuarios_unicos_conteo = resultados_usuario['usuario_num'].value_counts()
print(f"Total de usuarios en conjunto de test: {len(usuarios_unicos_conteo)}")
print(f"Distribución de muestras por usuario (primeros 10):")
print(usuarios_unicos_conteo.head(10))

# Obtener usuarios únicos en test
usuarios_unicos_test = usuarios_unicos_conteo.index.values

# Ajustar el umbral mínimo de muestras según los datos (reducir si es necesario)
min_muestras = 10  # Reducido de 20 a 10 para permitir más usuarios
min_digitos = 2    # Reducido de 3 a 2 para permitir más usuarios

print(f"\nUmbral mínimo de muestras por usuario: {min_muestras}")
print(f"Umbral mínimo de dígitos distintos por usuario: {min_digitos}")

for usuario in usuarios_unicos_test:
    # Filtrar datos para este usuario
    indices_usuario = resultados_usuario['usuario_num'] == usuario
    
    # Verificar si hay suficientes muestras
    if sum(indices_usuario) < min_muestras:
        print(f"Usuario {usuario}: insuficientes muestras ({sum(indices_usuario)}). Omitido.")
        continue
    
    # Datos específicos del usuario
    y_true_usuario = resultados_usuario.loc[indices_usuario, 'real'].values
    indices_indices_usuario = np.where(indices_usuario)[0]  # Obtener índices reales
    y_pred_prob_usuario = y_probs[indices_indices_usuario]
    
    # Verificar variedad de dígitos
    digitos_unicos = np.unique(y_true_usuario)
    if len(digitos_unicos) < min_digitos:
        print(f"Usuario {usuario}: insuficiente variedad de dígitos ({len(digitos_unicos)}). Omitido.")
        continue
    else:
        print(f"Usuario {usuario}: {sum(indices_usuario)} muestras, {len(digitos_unicos)} dígitos distintos: {digitos_unicos}")
    
    # Extraer confianza máxima para cada predicción
    max_confianzas = np.max(y_pred_prob_usuario, axis=1)
    
    # Etiqueta binaria: si la predicción fue correcta o no
    predicciones = np.argmax(y_pred_prob_usuario, axis=1)
    correctas = predicciones == y_true_usuario

    if len(np.unique(correctas)) < 2:
        print(f"Usuario {usuario}: solo una clase en etiquetas correctas. Omitido.")
        continue
    
    try:
        # Calcular EER
        eer, threshold, fpr, fnr = calculate_eer(correctas, max_confianzas)
        
        if eer is not None:
            # Guardar resultados
            eers_usuario.append(eer)
            usuarios_analizados.append(usuario)
            
            print(f"Usuario {usuario}: EER = {eer:.4f}, Umbral = {threshold:.4f}")
            
            # Guardar curva DET para este usuario
            plot_det_curve(
                fpr, fnr, eer=eer,
                label=f"Usuario {usuario} (EER={eer:.4f})",
                title=f"Curva DET para Usuario {usuario}",
                filename=f"curvas_det_usuarios/det_usuario_{usuario}.png"
            )
        else:
            print(f"No se pudo calcular EER para Usuario {usuario}")
    except Exception as e:
        print(f"Error al calcular EER para Usuario {usuario}: {e}")

# Ordenar EER por usuario (solo si hay datos)
if eers_usuario:
    df_eer_usuario = pd.DataFrame({
        'usuario_num': usuarios_analizados,
        'eer': eers_usuario
    })
    df_eer_usuario = df_eer_usuario.sort_values('eer')
    df_eer_usuario.to_csv('CNN/eer_por_usuario_ordenado.csv', index=False)

    # Graficar los 10 mejores y 10 peores usuarios (por EER)
    # Mejores usuarios (menor EER)
    if len(df_eer_usuario) > 10:
        plt.figure(figsize=(12, 6))
        mejores = df_eer_usuario.head(10)
        plt.bar(range(len(mejores)), mejores['eer'])
        plt.xlabel('Usuario')
        plt.ylabel('EER')
        plt.title('10 Usuarios con Menor EER')
        plt.xticks(range(len(mejores)), [f"U{u}" for u in mejores['usuario_num']], rotation=45)
        plt.grid(True, axis='y')
        plt.tight_layout()
        plt.savefig('CNN/mejores_usuarios_eer.png')
        plt.close()
        
        # Peores usuarios (mayor EER)
        plt.figure(figsize=(12, 6))
        peores = df_eer_usuario.tail(10)
        plt.bar(range(len(peores)), peores['eer'])
        plt.xlabel('Usuario')
        plt.ylabel('EER')
        plt.title('10 Usuarios con Mayor EER')
        plt.xticks(range(len(peores)), [f"U{u}" for u in peores['usuario_num']], rotation=45)
        plt.grid(True, axis='y')
        plt.tight_layout()
        plt.savefig('CNN/peores_usuarios_eer.png')
        plt.close()

print("\n----- ANÁLISIS COMPLETO -----")
print(f"Total dígitos analizados: {len(digitos)}")
print(f"Total usuarios analizados: {len(usuarios_analizados)}")

NumPy version: 1.26.0
TensorFlow version: 2.18.0
Verificación de asignación de IDs de usuario:
Número 0: 744 archivos
  - 101_session_1_0_002.png -> Usuario ID: 101
  - 101_session_1_0_004.png -> Usuario ID: 101
  - 101_session_1_0_006.png -> Usuario ID: 101
Número 1: 744 archivos
  - 101_session_1_1_002.png -> Usuario ID: 101
  - 101_session_1_1_004.png -> Usuario ID: 101
  - 101_session_1_1_006.png -> Usuario ID: 101
Número 2: 744 archivos
  - 101_session_1_2_002.png -> Usuario ID: 101
  - 101_session_1_2_004.png -> Usuario ID: 101
  - 101_session_1_2_006.png -> Usuario ID: 101
Número 3: 744 archivos
  - 101_session_1_3_002.png -> Usuario ID: 101
  - 101_session_1_3_004.png -> Usuario ID: 101
  - 101_session_1_3_006.png -> Usuario ID: 101
Número 4: 744 archivos
  - 101_session_1_4_002.png -> Usuario ID: 101
  - 101_session_1_4_004.png -> Usuario ID: 101
  - 101_session_1_4_006.png -> Usuario ID: 101
Número 5: 744 archivos
  - 101_session_1_5_002.png -> Usuario ID: 101
  - 101_session

c:\Users\BiDAlab\Proy_ASMI\ASMI_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

...

Usuario 130: 61 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 130: EER = 0.0500, Umbral = 0.8564
Usuario 122: 61 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 122: EER = 0.0167, Umbral = 0.7490
Usuario 147: 61 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 147: EER = 0.3333, Umbral = 0.9619
Usuario 203: 61 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 203: EER = 0.3750, Umbral = 0.9990
Usuario 187: 61 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 187: EER = 0.1000, Umbral = 0.8322
Usuario 118: 61 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 118: EER = 0.0172, Umbral = 0.7606
Usuario 196: 60 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 196: EER = 0.0000, Umbral = 0.6267
Usuario 120: 60 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 120: EER = 0.1667, Umbral = 0.8948
Usuario 148: 60 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 148: solo una clas

c:\Users\BiDAlab\Proy_ASMI\ASMI_env\lib\site-packages\scipy\interpolate\_interpolate.py:482: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Usuario 198: 60 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 198: EER = 0.0000, Umbral = 0.8989
Usuario 186: 60 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 186: EER = 0.0339, Umbral = 0.9763
Usuario 205: 59 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 205: EER = 0.0172, Umbral = 0.9673
Usuario 107: 59 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 107: EER = 0.0526, Umbral = 0.9066
Usuario 111: 59 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 111: solo una clase en etiquetas correctas. Omitido.
Usuario 135: 59 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 135: solo una clase en etiquetas correctas. Omitido.
Usuario 129: 58 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 129: EER = 0.0000, Umbral = nan
Usuario 169: 56 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 169: solo una clase en etiquetas correctas. Omitido.
Usuario 164: 56 muestras, 10 dígitos distinto

c:\Users\BiDAlab\Proy_ASMI\ASMI_env\lib\site-packages\scipy\interpolate\_interpolate.py:482: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo
c:\Users\BiDAlab\Proy_ASMI\ASMI_env\lib\site-packages\scipy\interpolate\_interpolate.py:482: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


Usuario 157: 55 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 157: EER = 0.0000, Umbral = 0.8191
Usuario 146: 55 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 146: solo una clase en etiquetas correctas. Omitido.
Usuario 104: 54 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 104: EER = 0.0980, Umbral = 0.9455
Usuario 199: 53 muestras, 10 dígitos distintos: [0 1 2 3 4 5 6 7 8 9]
Usuario 199: solo una clase en etiquetas correctas. Omitido.

----- ANÁLISIS COMPLETO -----
Total dígitos analizados: 10
Total usuarios analizados: 77
